In [ ]:
#import libraries
import numpy as np
import pprint
import scipy
import scipy.linalg
import pandas as pd
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.linear_model import Lasso
import matplotlib.pyplot as pl
import json
from tqdm import tqdm
from collections import defaultdict
np.set_printoptions(threshold=20)

In [ ]:
df = pd.read_csv("Books_rating.csv")

In [ ]:
df_book = pd.read_csv("books_data.csv")

In [ ]:
#drop_columns = ["description", "image", "previewLink", "publisher", "publishedDate", "infoLink"]

#books_data_cleaned = books_data.drop(drop_columns, axis=1)

#books_data_cleaned

category_array = df_book['categories'].unique()

category_array.size
#print('\n'.join(category_array.values))


In [ ]:
df

In [ ]:
df_book

In [ ]:
df.head()

In [ ]:
dfhead = df.head()

In [ ]:
#Test Preprocessing/Data Cleaning
drop_columns = ["Title", "Price", "profileName", "review/time", "review/summary", "review/helpfulness", "review/text"]

books_data_cleaned = dfhead.drop(drop_columns, axis=1)

books_data_cleaned

In [ ]:
df_ratings = df

In [ ]:
#Preprocessing/Data Cleaning
drop_columns = ["Price", "profileName", "review/time", "review/summary", "review/helpfulness", "review/text"]

books_ratings_cleaned = df_ratings.drop(drop_columns, axis=1)

books_ratings_cleaned

In [ ]:
#Create Dictionary Key for Unique ID:Book
book_dict = dict()
for row in tqdm(books_ratings_cleaned.itertuples()):
    if row[1] not in book_dict:
        book_dict[row[1]] = dict()

    book_dict[row[1]] = row[2]

In [ ]:
book_dict

In [ ]:
drop_columns = ["Title", "Price", "profileName", "review/time", "review/summary", "review/helpfulness", "review/text"]

books_ratings_cleaned = df_ratings.drop(drop_columns, axis=1)

books_ratings_cleaned

In [ ]:
books_ratings_cleaned['User_id'].isna()

sum(books_ratings_cleaned['User_id'].isna())

In [ ]:
books_ratings_cleaned = books_ratings_cleaned[books_ratings_cleaned['User_id'].notna()]

In [ ]:
books_ratings_cleaned

In [ ]:
#Create Dictionary Keys of the form     'User': {'Book ID': Rating, 'Book ID': Rating,...}
new_data = dict()
for row in tqdm(books_ratings_cleaned.sample(5000).itertuples()):
    if row[2] not in new_data:
        new_data[row[2]] = dict()

    new_data[row[2]][row[1]] = row[3]

In [ ]:
new_data

In [ ]:
#function to convert data frame to numpy matrix
def Convert_to_Matrix(matrix):
    #convert Dictionary Key into Sparse Matrix With users as Columns, Books as rows
    test = pd.DataFrame.from_dict(matrix)

    test.head()

    #take Matrix Transpose
    test_transpose = test.transpose()

    #populate NaN's as zeroes
    A_transpose = test_transpose.fillna(0)

    return A_transpose

#function to perform LU factorization on numpy matrix
def LU_Factorization(matrix):
    #A = PLU FACTORIZATION
    P, L, U = scipy.linalg.lu(matrix)

    A_test = np.matmul(P, np.matmul(L, U))

    #Test for Accuracy
    print(np.allclose(matrix, A_test))
    
    return P, L, U
    

In [ ]:
A = Convert_to_Matrix(new_data)

A.head()

In [ ]:
P, L, U = LU_Factorization(A)

In [ ]:
print(book_dict.get('B0009RJUKU'))

In [ ]:
# Some helper methods for getting subsets of users that have more data, reducing the sparcity of matrix
# when experimenting with the dataset
def get_most_frequent_users(count, source_frame):
    assert isinstance(count, int), "Argument must be an integer"
    assert count > 0, "Argument must be a positive integer"
    
    user_occurences = source_frame['User_id'].value_counts()
    
    assert count < user_occurences.count(), "Requested more data then we have...not tiny"
    
    return user_occurences.head(count).index.to_list()

def get_users_with_minimal_ratings(rated, source_frame):
    assert isinstance(rated, int), "Argument must be an integer"
    assert rated > 0, "Argument must be a positive integer"
    
    # Get a series mapping user ids to their occurences in the data frame
    user_occurences = source_frame['User_id'].value_counts()
    return user_occurences[user_occurences > rated].index.to_list()

def get_training_data(ratio: float, source_frame):
    assert isinstance(ratio, float), "Ratio must be a float"
    assert 0.0 < ratio < 1.0, "Ratio must be in [0, 1]"
    
    # we want to ensure that our test data is built from good users, users with
    # less then 3 ratings are considered un-testable.
    good_users = get_users_with_minimal_ratings(10, source_frame)
    
    # Randomize the order of ratings for users in our good users set.
    good_users_ratings = source_frame[source_frame['User_id'].isin(good_users)].sample(frac=1)
    
    total_count = good_users_ratings.shape[0]
    training_data_count = int(total_count * ratio)
    
    training_data = pivot_rating_to_user_frame(good_users_ratings[0:training_data_count])
    test_data = pivot_rating_to_user_frame(good_users_ratings[training_data_count:])
    
    return training_data, test_data

def pivot_rating_to_user_frame(source_frame):
    new_data = dict()
    for _, row in tqdm(source_frame.iterrows(), total=source_frame.shape[0]):
        (user_id, book_id, score) = row
        if user_id not in new_data:
            new_data[user_id] = defaultdict(lambda: np.nan)
        new_data[user_id][book_id] = score
        
    return pd.DataFrame.from_dict(new_data)

In [554]:
training_data, test_data = get_training_data(0.75, books_ratings_cleaned.sample(50000))

100%|█████████████████████████████████████████████████████████████████████████████| 299/299 [00:00<00:00, 11419.46it/s]


In [555]:
A = training_data.fillna(0)

A.head()

A

,B000Q751EM,B000I1S0YY,0689311435,0613919882,1588518671,1570020981,B0007FIG8Q,B0008B4ZP2,B000KU0Y88,B0009SRZEM,...,0486440265,B0006IU3N0,B0006AL5RG,B000NQ7GG2,0451519582,1578400317,0904724719,B0006AHG4C,B000G7CW6W,1401862926
A2F6N60Z96CAJI,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AJQ1S39GZBKUG,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A1JTG5X4VHJV27,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AFVQZQ8PW0L,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A2VKWLCNZF4ZVB,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A1LMBM1N4EXS5W,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A1K1JW1C5CUSUZ,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A2VE83MZF98ITY,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A1G37DFO8MQW0M,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A14OJS0VWMOSWO,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0


In [556]:
#finding similar users by using cosine similarity algorithm
def find_similar(user_1, k):
    allusers = A.values
    denominator1 = np.sqrt(sum([np.square(x) for x in user_1]))
    
    #performs cosine similarity algorithm on vectors (users)
    cosinesimilarity = [(user_1.name,1)]
    i=1
    for user in tqdm(allusers[1:]):
        numerator = [x*y for x,y in zip(user_1.values, user)]
        denominator2 = np.sqrt(sum([np.square(x) for x in user]))
        costheta = sum(numerator) / (denominator1 * denominator2)
        cosinesimilarity.append((A.index[i],costheta))
        i+=1
    
    #sort the results
    cosinesimilarity.sort(key = lambda x: x[1], reverse = True)
    
    #combine sorted results
    similar10users = cosinesimilarity[0:k]
    
    #output results of sorted similar users
    for i in range(0,k):
        print(similar10users[i])
    
    #place similar users into a data frame
    top10usersdf = pd.DataFrame()
    for user in similar10users:
        top10usersdf = top10usersdf.append(A.loc[user[0]])
    top10usersdf['costheta'] = [user[1] for user in similar10users]
    
    #to be used in calculation of distance/similarity for recommendation
    all_values = top10usersdf.values

    return top10usersdf

In [557]:
A_similar = find_similar(A.iloc[0],10)

A_similar

100%|█████████████████████████████████████████████████████████████████████████████████| 57/57 [00:00<00:00, 520.45it/s]

('A2F6N60Z96CAJI', 1)
('A2B1GZIRD6W646', 0.09545370481482404)
('AJQ1S39GZBKUG', 0.0)
('A1JTG5X4VHJV27', 0.0)
('AFVQZQ8PW0L', 0.0)
('A2VKWLCNZF4ZVB', 0.0)
('A1LMBM1N4EXS5W', 0.0)
('A1K1JW1C5CUSUZ', 0.0)
('A2VE83MZF98ITY', 0.0)
('A1G37DFO8MQW0M', 0.0)



C:\Users\14159\AppData\Local\Temp\ipykernel_14004\1301854071.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  top10usersdf = top10usersdf.append(A.loc[user[0]])
C:\Users\14159\AppData\Local\Temp\ipykernel_14004\1301854071.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  top10usersdf = top10usersdf.append(A.loc[user[0]])
C:\Users\14159\AppData\Local\Temp\ipykernel_14004\1301854071.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  top10usersdf = top10usersdf.append(A.loc[user[0]])
C:\Users\14159\AppData\Local\Temp\ipykernel_14004\1301854071.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  top10usersdf = top10usersdf.append

,B000Q751EM,B000I1S0YY,0689311435,0613919882,1588518671,1570020981,B0007FIG8Q,B0008B4ZP2,B000KU0Y88,B0009SRZEM,...,B0006IU3N0,B0006AL5RG,B000NQ7GG2,0451519582,1578400317,0904724719,B0006AHG4C,B000G7CW6W,1401862926,costheta
A2F6N60Z96CAJI,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.000000
A2B1GZIRD6W646,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.095454
AJQ1S39GZBKUG,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
A1JTG5X4VHJV27,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
AFVQZQ8PW0L,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
A2VKWLCNZF4ZVB,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
A1LMBM1N4EXS5W,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
A1K1JW1C5CUSUZ,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
A2VE83MZF98ITY,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
A1G37DFO8MQW0M,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000


In [ ]:
print(book_dict.get('B000N8T1L8'))